In [10]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
display(HTML("<style>.input_area pre {font-family: Consolas; font-size: 16pt; line-height: 140%;}</style>"))
display(HTML("<style>.output_area pre {font-family: Consolas; font-size: 16pt; line-height: 140%;}</style>"))

# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
import numpy as np
import os
import glob
import datetime
import shutil
from collections import defaultdict, Counter
from copy import deepcopy
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
plt.rc('font', family=font_name)
plt.rcParams['axes.unicode_minus'] = False

import matplotlib.patches as mpatches
import math

import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300

import statsmodels.stats.multitest as multest
import seaborn as sns
import scipy.stats as st

# calculate and save corr_mat

In [4]:
def get_z(r):
    if r==1:
        r=0.9999
    if r==-1:
        r=-0.9999
    return (math.log(1+r)-math.log(1-r))*0.5

def get_corr_unstack(df1,col_name):
    cor_df=df1.corr('pearson')
    cor_df=cor_df.unstack().to_frame()
    cor_df.columns=[col_name]
    cor_df.index.names=[None, None]
    return cor_df
    
file_names=glob.glob('result/TF_activity/effect/*.txt')  

mer_corr_integrated_df = pd.DataFrame()
for file_name in file_names:
    cell=file_name.split("\\")[-1].split('_')[2]
    print('#####',cell)
    
    anal_data=pd.read_table(file_name, sep='\t',engine='python')
    
    dataE=anal_data.loc[anal_data['effect']==1, [col for col in anal_data.columns if col!='effect']]
    dataNE=anal_data.loc[anal_data['effect']==0, [col for col in anal_data.columns if col!='effect']]
    
    corrE=get_corr_unstack(dataE, 'corrE')
    corrNE=get_corr_unstack(dataNE, 'corrNE')
    
    mer_corr=pd.concat([corrE,corrNE], axis=1)
    mer_corr=mer_corr.reset_index()
    
    mer_corr.columns=['g1','g2']+list(mer_corr.columns[2:])
    
    mer_corr['corr_diff']=mer_corr['corrE']-mer_corr['corrNE']

    mer_corr['corrEZ']=mer_corr['corrE'].map(get_z)
    mer_corr['corrNEZ']=mer_corr['corrNE'].map(get_z)
    
    mer_corr['Zobs(TCPS)']=mer_corr.apply(lambda row: (row['corrEZ']-row['corrNEZ'])/np.sqrt(1/(len(dataE)-3)+1/(len(dataNE)-3)), axis=1)
    mer_corr['p_value']=mer_corr['Zobs(TCPS)'].map(lambda x: st.norm.sf(abs(x))*2)

    mer_corr['FDR'] = multest.fdrcorrection(mer_corr['p_value'])[1]
    mer_corr['negLog10_FDR'] = mer_corr['FDR'].map(lambda x: -np.log10(x))

    ## save TTP score for each cell
    mer_corr.to_csv('result/TCPscore/TCPS_{}.txt'.format(cell),sep='\t')

    ## integration two cells
    mer_corr.set_index(['g1','g2'], inplace=True)
    
    for col in ['corrE','corrNE','corr_diff','corrEZ','corrNEZ','Zobs(TCPS)','negLog10_FDR']:
        mer_corr[col] = mer_corr[col].map(lambda x: '{:.3f}'.format(x))
        
    for col in ['p_value','FDR']:
        mer_corr[col] = mer_corr[col].map(lambda x: '{:.3e}'.format(x))
    
    mer_corr.rename(columns={'FDR':'FDR_{}'.format(cell)}, inplace=True)

    mer_corr_integrated_df=pd.concat([mer_corr_integrated_df, mer_corr], axis=1, sort=True)

mer_corr_integrated_df.reset_index(inplace=True)
mer_corr_integrated_df.to_csv('result/TCPscore/TCPS_both.txt',sep='\t')

##### A375
##### HT29


# remove duplicates of TCP scores

In [6]:
dupDel=open('result/TCPscore/TCPS_both_dupDel.txt','w+')

tfPair=set()

line_cnt=0
for line in open('result/TCPscore/TCPS_both.txt','r'):
    line_cnt+=1
    if line_cnt==1:
        _=dupDel.write(line)
        continue
        
    tf1,tf2=line.split('\t')[1:3]
    
    if tf1==tf2:
        continue
        
    if (tf1,tf2) not in tfPair:
        _=dupDel.write(line)
        tfPair.add((tf2,tf1))
    else:
        tfPair.remove((tf1,tf2))
    
    if line_cnt%10000==0:
        print(line_cnt)

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
